In [1]:
import json

In [5]:
# file = 'cse.json'
# file1 = "/Users/adebayobraimah/Downloads/drive-download-20240320T171548Z-001/sample_json_document.json"
file1 = 'sample.json'

In [12]:
# We have a JSON structure. Let's parse it and try to format it similarly to the sample.ergo file using Python.

# import json

# Load the JSON data from the uploaded file
json_file_path = file1 # '/mnt/data/sample.json'
with open(json_file_path, 'r') as file:
    json_data = json.load(file)

# This function will help to create a string similar to the ergo file format
def format_to_ergo(json_obj, parent_id):
    lines = []
    # We assign a unique ID to each JSON object
    oid = id(json_obj)
    # For each key, value in the JSON object, we decide how to format it based on the value's type
    for key, value in json_obj.items():
        if isinstance(value, dict):
            child_id = id(value)
            lines.append(f"{parent_id}['{key}' -> \\##{child_id}].")
            lines.extend(format_to_ergo(value, f"\\##{child_id}"))
        elif isinstance(value, list):
            # list_entries = ", ".join([f"{val if isinstance(val, int) else '\\##' + str(id(val))}" for val in value])
            list_entries = ", ".join(
                [
                    str(val) if isinstance(val, int) else f"\\##{id(val)}"
                    for val in value
                ]
            )

            lines.append(f"{parent_id}['{key}' -> [{list_entries}]].")
            for item in value:
                if isinstance(item, dict):
                    lines.extend(format_to_ergo(item, f"\\##{id(item)}"))
        else:
            lines.append(f"{parent_id}['{key}' -> {repr(value)}].")
    return lines

# Generate the formatted output
formatted_output = format_to_ergo(json_data, "\\##1")

# Joining all lines to form the final formatted string
formatted_output_string = "\n".join(formatted_output)
formatted_output_string

"\\##1['aaa' -> 1].\n\\##1['bbb' -> \\##4364044352].\n\\##4364044352['ccc' -> [1, 2, \\##4364044736]].\n\\##4364044736['111' -> 3].\n\\##4364044736['ppp' -> 4].\n\\##4364044352['ddd' -> 'ppp']."

In [9]:
# Fixing the backslash handling by using a different approach to construct the strings.

# Load the JSON data from the uploaded file
json_file_path = file1  # '/mnt/data/sample.json'
with open(json_file_path, "r") as file:
    json_data = json.load(file)

def format_to_ergo(json_obj, parent_id):
    lines = []
    lines.append("?- \@[add_json_root(\##1)]@\json.")
    # We assign a unique ID to each JSON object
    oid = id(json_obj)
    # For each key, value in the JSON object, we decide how to format it based on the value's type
    for key, value in json_obj.items():
        if isinstance(value, dict):
            child_id = id(value)
            lines.append(f"{parent_id}['{key}' -> \\\\##{child_id}].")
            lines.extend(format_to_ergo(value, f"\\\\##{child_id}"))
        elif isinstance(value, list):
            list_entries = []
            for val in value:
                if isinstance(val, int):
                    list_entries.append(str(val))
                else:
                    list_entries.append(f"\\\\##{id(val)}")
            lines.append(f"{parent_id}['{key}' -> [{', '.join(list_entries)}]].")
            for item in value:
                if isinstance(item, dict):
                    lines.extend(format_to_ergo(item, f"\\\\##{id(item)}"))
        else:
            lines.append(f"{parent_id}['{key}' -> {json.dumps(value)}].")
    lines.append(":- new_global_oid_scope.")
    lines.append("?- new_global_oid_scope.")
    return lines


# Generate the formatted output
formatted_output = format_to_ergo(json_data, "\\##1")
formatted_output

# Joining all lines to form the final formatted string
# formatted_output_string = "\n".join(formatted_output)
# formatted_output_string

['?- \\@[add_json_root(\\##1)]@\\json.',
 "\\##1['aaa' -> 1].",
 "\\##1['bbb' -> \\\\##4363603776].",
 '?- \\@[add_json_root(\\##1)]@\\json.',
 "\\\\##4363603776['ccc' -> [1, 2, \\\\##4364055232]].",
 '?- \\@[add_json_root(\\##1)]@\\json.',
 "\\\\##4364055232['111' -> 3].",
 "\\\\##4364055232['ppp' -> 4].",
 ':- new_global_oid_scope.',
 '?- new_global_oid_scope.',
 '\\\\##4363603776[\'ddd\' -> "ppp"].',
 ':- new_global_oid_scope.',
 '?- new_global_oid_scope.',
 ':- new_global_oid_scope.',
 '?- new_global_oid_scope.']